In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=4b5bf30d1c2f4887bac60273acea4fa1 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=7da7713a716346e28426423aae54ac75 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.54it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.93it/s]

Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/bd5872a6f3c940e8ad75932b5d86a544.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=15944dfbb27148cab91f4d88f7642190 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_3584: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_3584
override True
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

## Sync to high side

In [10]:
low_state = client_low.sync.get_state()

high_state = client_high.sync.get_state()

SyftInfo: Node Landed!


In [11]:
low_state

syft.service.sync.sync_state.SyncState

In [12]:
diff_state = compare_states(low_state, high_state)

diff_state

syft.service.sync.diff_state.NodeDiff

In [13]:
# syft absolute
# state.objs_to_sync
from syft.client.syncing import resolve

resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

╭─ Request(ff55aa09255c4895a6ded75bb5d40210) - State: NEW ──────────────────────────────────╮
│                                                                                           │
│  ╭─ Low side ──────────────────────────────────────────────────────────────────────────╮  │
│  │ ┌─────────────────────────────────────────────────────────────────────────────────┐ │  │
│  │ │ NEW                                                                             │ │  │
│  │ │                                                                                 │ │  │
│  │ │ class Request:                                                                  │ │  │
│  │ │ ‎ ‎ ‎ ‎ request_time = 2024-02-29 17:18:04                                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ updated_at = None                                                           │ │  │
│  │ │ ‎ ‎ ‎ ‎ status = RequestStatus.PENDING                                              │ │  │
│  │ │ ‎ ‎ ‎ ‎ changes = [                                                                 │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                    │ │  │
│  │ │ ‎ ‎ ‎ ‎ ]                                                                           │ │  │
│  │ │ ‎ ‎ ‎ ‎ requesting_user_verify_key =                                                │ │  │
│  │ │ aebd804f4435afe078768aa67fb2011a9a918586ae6e62c7be1ab0e4aed2214e                │ │  │
│  │ └─────────────────────────────────────────────────────────────────────────────────┘ │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ─────────────────────────────────────────────────────────────────────────╮  │
│  │ None                                                                                │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

╭─ UserCodeStatusCollection(3e8bb5a363ef42e58a922d0990465734) - State: NEW ────────────╮
     │                                                                                      │
     │  ╭─ Low side ─────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ NEW                                                                        │ │  │
     │  │ │                                                                            │ │  │
     │  │ │ class UserCodeStatusCollection:                                            │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ────────────────────────────────────────────────────────────────────╮  │
     │  │ None                                                                           │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                      │
     ╰──────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from low side


╭─ UserCode(a3283fa5e49b4989b8c746d162e0da88) - State: NEW ─────────────────────────────────╮
│                                                                                           │
│  ╭─ Low side ──────────────────────────────────────────────────────────────────────────╮  │
│  │ ┌─────────────────────────────────────────────────────────────────────────────────┐ │  │
│  │ │ NEW                                                                             │ │  │
│  │ │                                                                                 │ │  │
│  │ │ class UserCode:                                                                 │ │  │
│  │ │ ‎ ‎ ‎ ‎ service_func_name = compute_mean                                            │ │  │
│  │ │ ‎ ‎ ‎ ‎ input_owners = [                                                            │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ test_l                                                                  │ │  │
│  │ │ ‎ ‎ ‎ ‎ ]                                                                           │ │  │
│  │ │ ‎ ‎ ‎ ‎ code_status = [                                                             │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ Node: test_l, Status: pending                                           │ │  │
│  │ │ ‎ ‎ ‎ ‎ ]                                                                           │ │  │
│  │ │ ‎ ‎ ‎ ‎ worker_pool_name = default-pool                                             │ │  │
│  │ └─────────────────────────────────────────────────────────────────────────────────┘ │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ─────────────────────────────────────────────────────────────────────────╮  │
│  │ None                                                                                │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from low side


In [14]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[syft.service.request.request.Request, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode],
  update_objs=[],
  delete_objs=[]
)


In [15]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [16]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 3 items

# Run code high and sync back result

In [17]:
data_high = client_high.datasets[0].assets[0]

In [18]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = f828b400cf194d70ae1689f629ce9ea7
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [19]:
# wait for the result
job_high.wait().get()

override True


29/02/24 17:18:12 FUNCTION LOG (f828b400cf194d70ae1689f629ce9ea7): Computing mean...


17.0

In [20]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [21]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [22]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: de94bbe9bcf143a5add4ba696901239a as aebd804f4435afe078768aa67fb2011a9a918586ae6e62c7be1ab0e4aed2214e] de94bbe9bcf143a5add4ba696901239a
New result 17.0
Job(f828b400cf194d70ae1689f629ce9ea7) Setting new result de94bbe9bcf143a5add4ba696901239a -> de94bbe9bcf143a5add4ba696901239a


SyftSuccess: Request submitted for updating result.

In [23]:
request.code.output_history[0]

```python
class ExecutionOutput:
  id: str = 9fad167a8a3744a69b471210c63d6e8d
  created_at: str = 2024-02-29 17:17:49
  user_code_id: str = a3283fa5e49b4989b8c746d162e0da88
  job_id: str = f828b400cf194d70ae1689f629ce9ea7
  output_ids: str = [<LineageID: de94bbe9bcf143a5add4ba696901239a - 178624147843924223>]

```

In [24]:
# accept_res = request.accept_by_depositing_result(job_info.result.get())

# assert isinstance(accept_res, SyftSuccess)
# accept_res

In [25]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [26]:
client_high.jobs[0]

```python
class Job:
    id: UID = f828b400cf194d70ae1689f629ce9ea7
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```

In [27]:
# syft absolute

In [28]:
# syft absolute

In [29]:
# x = deserialize(serialize(UserCodeStatusCollection(status_dict=[])))

In [30]:
# x.syft_eq

In [31]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

high_state

syft.service.sync.sync_state.SyncState

In [32]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

syft.service.sync.diff_state.NodeDiff

In [36]:
resolved_state_low, resolved_state_high = resolve(diff_state_2, decision="high")

╭─ ExecutionOutput(9fad167a8a3744a69b471210c63d6e8d) - State: NEW ──────────────────────────╮
│                                                                                           │
│  ╭─ Low side ──────────────────────────────────────────────────────────────────────────╮  │
│  │ None                                                                                │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ─────────────────────────────────────────────────────────────────────────╮  │
│  │ ┌─────────────────────────────────────────────────────────────────────────────────┐ │  │
│  │ │ NEW                                                                             │ │  │
│  │ │                                                                                 │ │  │
│  │ │ class ExecutionOutput:                                                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ created_at = 2024-02-29 17:17:49                                            │ │  │
│  │ │ ‎ ‎ ‎ ‎ user_code_id = a3283fa5e49b4989b8c746d162e0da88                             │ │  │
│  │ │ ‎ ‎ ‎ ‎ job_id = f828b400cf194d70ae1689f629ce9ea7                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ output_ids = [                                                              │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ de94bbe9bcf143a5add4ba696901239a                                        │ │  │
│  │ │ ‎ ‎ ‎ ‎ ]                                                                           │ │  │
│  │ └─────────────────────────────────────────────────────────────────────────────────┘ │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│                                                                                           │
╰───────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Job(f828b400cf194d70ae1689f629ce9ea7) - State: NEW ─────────────────────────────────╮
     │                                                                                      │
     │  ╭─ Low side ─────────────────────────────────────────────────────────────────────╮  │
     │  │ None                                                                           │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ NEW                                                                        │ │  │
     │  │ │                                                                            │ │  │
     │  │ │ class Job:                                                                 │ │  │
     │  │ │ ‎ ‎ ‎ ‎ id = f828b400cf194d70ae1689f629ce9ea7                                  │ │  │
     │  │ │ ‎ ‎ ‎ ‎ result = 17.0                                                          │ │  │
     │  │ │ ‎ ‎ ‎ ‎ resolved = True                                                        │ │  │
     │  │ │ ‎ ‎ ‎ ‎ progress =                                                             │ │  │
     │  │ │ ‎ ‎ ‎ ‎ creation_time = 2024-02-29 17:18:09.605261                             │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                      │
     ╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─ SyftLog(b3163188267c459cbb52e24e03ecedf8) - State: NEW ────────────────────────╮
          │                                                                                 │
          │  ╭─ Low side ────────────────────────────────────────────────────────────────╮  │
          │  │ None                                                                      │  │
          │  ╰───────────────────────────────────────────────────────────────────────────╯  │
          │  ╭─ High side ───────────────────────────────────────────────────────────────╮  │
          │  │ ┌───────────────────────────────────────────────────────────────────────┐ │  │
          │  │ │ NEW                                                                   │ │  │
          │  │ │                                                                       │ │  │
          │  │ │ class SyftLog:                                                        │ │  │
          │  │ │ ‎ ‎ ‎ ‎ stdout = Computing mean...                                        │ │  │
          │  │ │                                                                       │ │  │
          │  │ │ ‎ ‎ ‎ ‎ stderr =                                                          │ │  │
          │  │ └───────────────────────────────────────────────────────────────────────┘ │  │
          │  ╰───────────────────────────────────────────────────────────────────────────╯  │
          │                                                                                 │
          ╰─────────────────────────────────────────────────────────────────────────────────╯

╭─ UserCode(a3283fa5e49b4989b8c746d162e0da88) - State: DIFF ───────────────────────────╮
     │                                                                                      │
     │  ╭─ Low side ─────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ class UserCode:                                                            │ │  │
     │  │ │ ‎ ‎ ‎ ‎ node_uid=8a1c04544655402190588aec30079bc3,                             │ │  │
     │  │ │ ‎ ‎ ‎ ‎ status_link=UserCodeStatusCollection: 3e8bb5a363ef42e58a922d0990465734 │ │  │
     │  │ │ @ Node 8a1c04544655402190588aec30079bc3                                    │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ class UserCode:                                                            │ │  │
     │  │ │ ‎ ‎ ‎ ‎ node_uid=8212e6797fde4c3fba4fc53ab555a886,                             │ │  │
     │  │ │ ‎ ‎ ‎ ‎ status_link=UserCodeStatusCollection: 3e8bb5a363ef42e58a922d0990465734 │ │  │
     │  │ │ @ Node 8212e6797fde4c3fba4fc53ab555a886                                    │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                      │
     ╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─ NumpyScalarObject(de94bbe9bcf143a5add4ba696901239a) - State: NEW ───────────────────╮
     │                                                                                      │
     │  ╭─ Low side ─────────────────────────────────────────────────────────────────────╮  │
     │  │ None                                                                           │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ Pointer:                                                                   │ │  │
     │  │ │ 17.0                                                                       │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                      │
     ╰──────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from high side


╭─ Request(ff55aa09255c4895a6ded75bb5d40210) - State: DIFF ─────────────────────────────────╮
│                                                                                           │
│  ╭─ Low side ──────────────────────────────────────────────────────────────────────────╮  │
│  │ ┌─────────────────────────────────────────────────────────────────────────────────┐ │  │
│  │ │ class Request:                                                                  │ │  │
│  │ │ ‎ ‎ ‎ ‎ updated_at=None,                                                            │ │  │
│  │ │ ‎ ‎ ‎ ‎ node_uid=8a1c04544655402190588aec30079bc3,                                  │ │  │
│  │ │ ‎ ‎ ‎ ‎ changes=[                                                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                    │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ ],                                                                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ changes=[                                                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                    │ │  │
│  │ │ ‎ ‎ ‎ ‎ ],                                                                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ history=[                                                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus                               │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus                               │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus                               │ │  │
│  │ │ ‎ ‎ ‎ ‎ ],                                                                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ history=[                                                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ ]                                                                           │ │  │
│  │ └─────────────────────────────────────────────────────────────────────────────────┘ │  │
│  ╰─────────────────────────────────────────────────────────────────────────────────────╯  │
│  ╭─ High side ─────────────────────────────────────────────────────────────────────────╮  │
│  │ ┌─────────────────────────────────────────────────────────────────────────────────┐ │  │
│  │ │ class Request:                                                                  │ │  │
│  │ │ ‎ ‎ ‎ ‎ updated_at=2024-02-29 17:18:16,                                             │ │  │
│  │ │ ‎ ‎ ‎ ‎ node_uid=8212e6797fde4c3fba4fc53ab555a886,                                  │ │  │
│  │ │ ‎ ‎ ‎ ‎ changes=[                                                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ UserCodeStatusChange                                                    │ │  │
│  │ │ ‎ ‎ ‎ ‎ ],                                                                          │ │  │
│  │ │ ‎ ‎ ‎ ‎ changes=[                                                                   │ │  │
│  │ │ ‎ ‎ ‎ ‎ [38;2;0;0;0;48;2;248;248;248

╭─ UserCodeStatusCollection(3e8bb5a363ef42e58a922d0990465734) - State: DIFF ───────────╮
     │                                                                                      │
     │  ╭─ Low side ─────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ class UserCodeStatusCollection:                                            │ │  │
     │  │ │ ‎ ‎ ‎ ‎ status_dict=None,                                                      │ │  │
     │  │ │ ‎ ‎ ‎ ‎ status=(<UserCodeStatus.PENDING: 'pending'>, '')                       │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │  ╭─ High side ────────────────────────────────────────────────────────────────────╮  │
     │  │ ┌────────────────────────────────────────────────────────────────────────────┐ │  │
     │  │ │ class UserCodeStatusCollection:                                            │ │  │
     │  │ │ ‎ ‎ ‎ ‎ status_dict=None,                                                      │ │  │
     │  │ │ ‎ ‎ ‎ ‎ status=(<UserCodeStatus.APPROVED: 'approved'>, '')                     │ │  │
     │  │ └────────────────────────────────────────────────────────────────────────────┘ │  │
     │  ╰────────────────────────────────────────────────────────────────────────────────╯  │
     │                                                                                      │
     ╰──────────────────────────────────────────────────────────────────────────────────────╯

Decision: Syncing all objects from high side


In [43]:
print(resolved_state_low)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog, Pointer:
17.0],
  update_objs=[syft.service.code.user_code.UserCode, syft.service.request.request.Request, {NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}],
  delete_objs=[]
)


In [41]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 7 items

In [42]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [44]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [45]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False
output_history [syft.service.output.output_service.ExecutionOutput] aefe279a99614cea9d589d4b00eaa859
output_history [syft.service.output.output_service.ExecutionOutput] aefe279a99614cea9d589d4b00eaa859


17.0

In [46]:
res_low.get()

17.0

<UID: de94bbe9bcf143a5add4ba696901239a>

In [58]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [59]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False
output_history [syft.service.output.output_service.ExecutionOutput] aefe279a99614cea9d589d4b00eaa859
output_history [syft.service.output.output_service.ExecutionOutput] aefe279a99614cea9d589d4b00eaa859


17.0

In [60]:
res_low.get()

17.0

In [61]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [62]:
private_high.mean()

17.0

In [63]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = f828b400cf194d70ae1689f629ce9ea7
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Log b3163188267c459cbb52e24e03ecedf8 not available
JOB COMPLETED
    
```

In [64]:
low_state = client_low.sync.get_state()

In [65]:
job_low.wait().get()

17.0

In [66]:
job_low.logs()

Log b3163188267c459cbb52e24e03ecedf8 not available


In [67]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)

## TODO

- Hard-deleting objects (in a clean way)
    - We *should* be able to delete without messing up a node state, because we make a single choice for a whole hierarchy 
    - What happens if we delete a queued/running job?
- Soft-delete/archive objects
    - More work, new flag on every object in every stash
- 